# Web Scraping in Python

In this appendix lecture we'll go over how to scrape information from the web using Python. 

#####We'll go to a website, decide what information we want, see where and how it is stored, then scrape it and set it as a pandas DataFrame!

####Some things you should consider before web scraping a website:

1.) You should check a site's terms and conditions before you scrape them. 

2.) Space out your requests so you don't overload the site's server, doing this could get you blocked.

3.) Scrapers break after time - web pages change their layout all the time, you'll more than likely have to rewrite your code. 

4.) Web pages are usually inconsistent, more than likely you'll have to clean up the data after scraping it.

5.) Every web page and situation is different, you'll have to spend time configuring your scraper.

####To learn more about HTML I suggest theses two resources:

[W3School](http://www.w3schools.com/html/)

[Codecademy](http://www.codecademy.com/tracks/web)


####There are three modules we'll need in addition to python are:

1.) BeautifulSoup, which you can download by typing: *pip install beautifulsoup4* or *conda install beautifulsoup4* (for the Anaconda distrbution of Python) in your command prompt.

2.) lxml , which you can download by typing: *pip install lxml* or *conda install lxml* (for the Anaconda distrbution of Python) in your command prompt.

3.) requests,  which you can download by typing: *pip install requests* or *conda install requests* (for the Anaconda distrbution of Python) in your command prompt.



We'll start with our imports:

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import pandas as pd
from pandas import Series,DataFrame

For our quick web scraping tutorial, we'll look at some legislative reports from the University of California Web Page. Feel free to experiment with other webpages, but remember to be cautious and respectful in what you scrape and how often you do it. Always check the legality of a web scraping job.

Let's go ahead and set the url.

In [4]:
url = 'http://www.ucop.edu/operating-budget/budgets-and-reports/legislative-reports/2013-14-legislative-session.html'

Now let's go ahead and set up requests to grab content form the url, and set it as a Beautiful Soup object.

In [5]:
# Request content from web page
result = requests.get(url)
c = result.content

# Set as Beautiful Soup Object
soup = BeautifulSoup(c)

Now we'll use Beautiful Soup to search for the table we want to grab!

In [6]:
# Go to the section of interest
summary = soup.find("div",{'class':'list-land','id':'content'})

# Find the tables in the HTML
tables = summary.find_all('table')


Now we need to use Beautiful Soup to find the table entries. A 'td' tag  defines a standard cell in an HTML table. The 'tr' tag defines a row in an HTML table.

We'll parse through our tables object and try to find each cell using the findALL('td') method.

There are tons of options to use with findALL in beautiful soup. You can read about them [here](http://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all).

In [19]:
# Set up empty data list
data = []

# Set rows as first indexed object in tables with a row
rows = tables[0].findAll('tr')

# now grab every HTML cell in every row
for tr in rows:
    cols = tr.findAll('td')
    # Check to see if text is in the row
    for td in cols:
        text = td.find(text=True) 
        print text,
        data.append(text)
    

1 08/01/13 2013-14 (EDU 92495) Proposed Capital Outlay Projects (2013-14 only) (pdf) 2 09/01/13 2014-15  (EDU 92495) Proposed Capital Outlay Projects (pdf) 3 11/01/13 Utilization of Classroom and Teaching Laboratories (pdf) 4 11/01/13 Instruction and Research Space Summary & Analysis (pdf) 5 11/15/13 Statewide Energy Partnership Program (pdf) 6 11/30/13 2013-23 Capital Financial Plan (pdf) 7 11/30/13 Projects Savings Funded from Capital Outlay Bond Funds (pdf) 8 12/01/13 Streamlined Capital Projects Funded from Capital (pdf) 9 01/01/14 Annual General Obligation Bonds Accountability (pdf) 10 01/01/14 Small Business Utilization (pdf) 11 01/01/14 Institutional Financial Aid Programs - Preliminary report (pdf) 12 01/10/14 Summer Enrollment (pdf) 13 01/15/14 Contracting Out for Services at Newly Developed Facilities (pdf) 14 03/01/14 Performance Measures (pdf) 15 03/01/14 Entry Level Writing Requirement (pdf) 16 03/31/14 Annual Report on Student Financial Support (pdf) 17 04/01/14 Unique St

Let's see what the data list looks like

In [20]:
data

[u'1',
 u'08/01/13',
 u'2013-14 (EDU 92495) Proposed Capital Outlay Projects (2013-14 only) (pdf)',
 u'2',
 u'09/01/13',
 u'2014-15\xa0 (EDU 92495) Proposed Capital Outlay Projects (pdf)',
 u'3',
 u'11/01/13',
 u'Utilization of Classroom and Teaching Laboratories (pdf)',
 u'4',
 u'11/01/13',
 u'Instruction and Research Space Summary & Analysis (pdf)',
 u'5',
 u'11/15/13',
 u'Statewide Energy Partnership Program (pdf)',
 u'6',
 u'11/30/13',
 u'2013-23 Capital Financial Plan (pdf)',
 u'7',
 u'11/30/13',
 u'Projects Savings Funded from Capital Outlay Bond Funds (pdf)',
 u'8',
 u'12/01/13',
 u'Streamlined Capital Projects Funded from Capital (pdf)',
 u'9',
 u'01/01/14',
 u'Annual General Obligation Bonds Accountability (pdf)',
 u'10',
 u'01/01/14',
 u'Small Business Utilization (pdf)',
 u'11',
 u'01/01/14',
 u'Institutional Financial Aid Programs - Preliminary report (pdf)',
 u'12',
 u'01/10/14',
 u'Summer Enrollment (pdf)',
 u'13',
 u'01/15/14',
 u'Contracting Out for Services at Newly De

Now we'll use a for loop to go through the list and grab only the cells with a pdf file in them, we'll also need to keep track of the index to set up the date of the report.

In [21]:
# Set up empty lists
reports = []
date = []

# Se tindex counter
index = 0

# Go find the pdf cells
for item in data:
    if 'pdf' in item:
        # Add the date and reports
        date.append(data[index-1])
        
        # Get rid of \xa0
        reports.append(item.replace(u'\xa0', u' '))
                    
    index += 1

You'll notice a line to take care of '\xa0 ' This is due to a unicode error that occurs if you don't do this. Web pages can be messy and inconsistent and it is very likely you'll have to do some research to take care of problems like these.

Here's the link I used to solve this particular issue: [StackOverflow Page](http://stackoverflow.com/questions/10993612/python-removing-xa0-from-string)

Now all that is left is to organize our data into a pandas DataFrame!

In [14]:
# Set up Dates and Reports as Series
date = Series(date)
reports = Series(reports)

In [15]:
# Concatenate into a DataFrame
legislative_df = pd.concat([date,reports],axis=1)

In [16]:
# Set up the columns
legislative_df.columns = ['Date','Reports']

In [18]:
# Show the finished DataFrame
legislative_df

,Date,Reports
0,08/01/13,2013-14 (EDU 92495) Proposed Capital Outlay Pr...
1,09/01/13,2014-15 (EDU 92495) Proposed Capital Outlay P...
2,11/01/13,Utilization of Classroom and Teaching Laborato...
3,11/01/13,Instruction and Research Space Summary & Analy...
4,11/15/13,Statewide Energy Partnership Program (pdf)
5,11/30/13,2013-23 Capital Financial Plan (pdf)
6,11/30/13,Projects Savings Funded from Capital Outlay Bo...
7,12/01/13,Streamlined Capital Projects Funded from Capit...
8,01/01/14,Annual General Obligation Bonds Accountability...
9,01/01/14,Small Business Utilization (pdf)


There are other less intense options for web scraping:

Check out these two companies:

https://import.io/

https://www.kimonolabs.com/


## Good Job!